In [ ]:
import pickle
import networkx as nx

def load_graph():
    G = nx.read_gpickle("graph.gpickle")
    return G

def getNodesByAtt(G, attDict):
    ND = G.nodes(data=True)
    nodesWhere = []
    for (n, d) in ND:
        match = False
        for att in attDict:
            if d[att] == attDict[att]:
                match = True
            else:
                match = False
        if match:
            nodesWhere.append(n)
    return nodesWhere

def getNodeByName(G, name):
    node = getNodesByAtt(G, {'name' : name})
    if node: return node[0]

def checkForNode(G, name):
    if getNodeByName(G, name):
        return False
    return true

def setNewID(G):
    x = len(G) + 1
    return x

def add_team(name, parent, relationship, description='', pulse='', confluence='', github=''):
    #TODO check for team node of same name
    G = load_graph()
    if checkForNode(G, name):
        G.add_node(setNewID(G),name=name, type='Team', description=description, pulse=pulse, confluence=confluence, github=github)
        node = getNodeByName(G, name)
        parent = getNodeByName(G, name)
        add_link(parent, node, relationship=relationship, G=G)


def add_okr(setNewID,name, parent_okr, parent_team):
    G = load_graph()
    if checkForNode(G, name):
        G.add_node(setNewID(G),name=name, type='OKR')
        node = G.nodes[name]
        add_link(parent_okr, node, relationship='Assigned_to', G=G)
        add_link(parent_team, node, relationship='Assigned_to', G=G)

def add_key_result(name, parent_okr):
    G = load_graph()
    if checkForNode(G, name):
        G.add_node(setNewID(G),name=name, type='Key_Result')
        node = G.nodes[name]
        add_link(parent_okr, node, relationship='Assigned_to', G=G)

def add_member(eid, parent_team):
    #TODO check for team node of same name
    G = load_graph()
    if checkForNode(G, name):
        G.add_node(setNewID(G),name=eid, type='Member')
        node = G.nodes[eid]
        add_link(parent_okr, node, relationship='Assigned_to', G=G)

def del_team(name):
    return true

def del_okr(name):
    return true

def del_key_result(name):
    return true

def del_member(name):
    return true

def add_link(parent, child, relationship, G):
    G.add_edge(parent, child, relationship=relationship)
    nx.write_gpickle(G, "graph.gpickle")

def del_link(parent, child, G):
    return true


In [ ]:
from networkx.readwrite import json_graph

def makeJSONgraph(G, root_node):
    del_nodes = []
    for (n, d) in G.nodes(data=True):
#         print(n)
#         print(d)
#         print(d['type'])
        if d['type'] not in ['Root', 'Team']:
            del_nodes.append(n)
    G.remove_nodes_from(del_nodes)
    data = json_graph.tree_data(G,root=root_node)
    datastr = json.dumps(data).replace("children", "_children").replace("NaN", '""').replace("id","name")
    data = json.loads(datastr)
    data['children'] = data.pop('_children')
    return data

In [ ]:
#remove nan nodes
del_nodes = []
for n in N:
    if  isinstance(n, float):
        del_nodes.append(n)
# G.remove_nodes_from(del_nodes)

In [ ]:
#remove orphan (degree 0) nodes
for n in N:
    if  G.degree[n] == 0:
        del_nodes.append(n)
G.remove_nodes_from(del_nodes)
print(del_nodes)
N = G.nodes()

In [ ]:
def link_nodes(graph, parent_node, node_list):
    x = zip([parent_node]*len(node_list),( node for node in node_list))
    graph.add_edges_from(x)

In [ ]:
def drawShellGraph(G, fileName='graph', groups=[], save=False, node_size=100, font_size=8):
    if len(groups)==0:
        # get unique groups
        groups = set(nx.get_node_attributes(G,'type').values())
    mapping = dict(zip(sorted(groups),count()))
    nodes = G.nodes()
    colors = [mapping[G.node[n]['type']] for n in nodes]
    shells = []#['MBR', 'Section','report', 'data source', 'person' ]
    for group in groups:
        shell = []
        if isinstance(group, str):
            for n,d in G.nodes(data=True):
                if d['type']==group:
                    shell.append(n)
            shells.append(shell)

    # drawing nodes and edges separately so we can capture collection for colobar
    pos = nx.shell_layout(G, shells)

    ec = nx.draw_networkx_edges(G, pos, alpha=0.2, arrows=False, width=.5)
    nc = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors, 
                                node_size=node_size, cmap=plt.cm.jet)
    lc = nx.draw_networkx_labels(G, pos, font_size=font_size)
    # plt.colorbar(nc)
    plt.axis('off')
    if save:
        plt.savefig("images/shell_"+fileName+".png", dpi=2000)

In [ ]:
def drawGraph(G, fileName='graph', groups=[], save=False, node_size=100, font_size=8):
    if len(groups)==0:
        # get unique groups
        groups = set(nx.get_node_attributes(G,'type').values())
    # groups = ['MBR', 'Section', 'report', 'data source', 'person']
    mapping = dict(zip(sorted(groups),count()))
    nodes = G.nodes()
    colors = [mapping[G.node[n]['type']] for n in nodes]

    # pos = nx.kamada_kawai_layout(G)
    pos = nx.spring_layout(G)

    ec = nx.draw_networkx_edges(G, pos, alpha=0.2, arrows=False, width=.5)
    nc = nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color=colors, 
                                node_size=node_size, cmap=plt.cm.jet)
    lc = nx.draw_networkx_labels(G, pos, font_size=font_size)
    plt.axis('off')
    if save:
        plt.savefig("images/"+fileName+".png", dpi=2000)

In [ ]:
def getNodeByDegreeAttr(G, degree=1, relate='=', attDict=[], direction="all"):
    ops = {'>': operator.gt,
               '<': operator.lt,
               '>=': operator.ge,
               '<=': operator.le,
               '=': operator.eq}
    points = {'all': G.degree,
                 'in': G.in_degree,
                 'out': G.out_degree}
    ND = G.nodes(data=True)
    nodeMatch = []
    attSearch = True if len(attDict) > 0 else False
    for (n,d) in ND:
        if ops[relate](points[direction](n), degree):
            if not attSearch:
                nodeMatch.append(n)
            else:
                match = False
                for att in attDict:
                    if d[att] == attDict[att]:
                        match = True
                    else:
                        match = False
                if match:
                    nodeMatch.append(n)
    return nodeMatch

In [ ]:
def leafCountByType(G):
    N = getNodeByDegreeAttr(G, degree=1, relate='=')
    cnts = dict(Counter([G.node[n]['type'] for n in N]))
    return cnts

In [ ]:
def getSubGraph(G, source='MBR'):
#     t = nx.bfs_edges(G, source=source, reverse=True)
#     T = nx.DiGraph((x, y) for (x, y) in list(t))
# #     nx.draw_kamada_kawai(T, with_labels=True, font_size=8)
#     return T
    nSet = set()
    for e in list(nx.bfs_edges(G, source=source, reverse=True)):
        for n in e:
            nSet.add(n)
    return G.subgraph(nSet)

In [ ]:
def makeSubGraph(G, nodes, target='MBR'):
    nSet = set()
    for node in nodes:
        for e in list(nx.all_simple_paths(G, node, target)):
            for n in e:
                nSet.add(n)
    return G.subgraph(nSet)